<a href="https://colab.research.google.com/github/Ray7788/FT5005-Group6/blob/main/Cleaned%20Data/EBITDA_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab\ Notebooks/FT5005/Cleaned\ Data/

Data Cleaning for **Revenue**
========================
This notebook contains the data cleaning process for the revenue dataset used in the project.

The input dataset is a CSV file which is merged and preprocessed from the FT5005 professor's official dataset and self-explored dataset. The file used for this process is `revenue_preprocess.csv`.

In [17]:
import pandas as pd
import numpy as np
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [18]:
df = pd.read_csv("revenue_preprocess.csv")
print(df.columns.tolist())
df.head()

['tic', 'datafqtr', 'cshoq', 'cshtrq', 'capxy', 'oiadpq', 'niq', 'xoprq', 'teqq', 'atq', 'epsfiq', 'cshoq.1', 'cshtrq.1', 'capxy.1', 'oiadpq.1', 'niq.1', 'xoprq.1', 'teqq.1', 'atq.1', 'epsfiq.1', 'gmrmexpq', 'gmrmrevq', 'hbinvfhq', 'hbinvludq', 'hbinvtq', 'hbloq', 'conm', 'revtq', 'sentiment_x', 'sentiment_y', 'SALE PRICE', 'Real GDP SA(billion)', 'M2 SA(billion)', 'M2-M1 Growth Gap(%)', 'M2V', 'Prime Rate(%)']


,tic,datafqtr,cshoq,cshtrq,capxy,oiadpq,niq,xoprq,teqq,atq,...,conm,revtq,sentiment_x,sentiment_y,SALE PRICE,Real GDP SA(billion),M2 SA(billion),M2-M1 Growth Gap(%),M2V,Prime Rate(%)
0,NLP,2010Q1,10.666,-2.568,10.666,11.450,263614.0,11.450,263614.0,63.433,...,263614.0,63.433,0.0,63.433,0.0,309.571,0.209,309.571,0.209,-0.23
1,NLP,2010Q2,10.666,-2.608,10.666,12.143,235341.0,12.143,235341.0,59.999,...,235341.0,59.999,0.0,59.999,0.0,304.493,-0.307,304.493,-0.307,-0.23
2,NLP,2010Q3,10.666,-3.197,10.666,11.950,159175.0,11.950,159175.0,55.925,...,159175.0,55.925,0.0,55.925,0.0,299.536,0.336,299.536,0.336,-0.28
3,NLP,2010Q4,10.666,-3.023,10.666,11.716,163338.0,11.716,163338.0,52.167,...,163338.0,52.167,0.0,52.167,0.0,328.566,0.642,328.566,0.642,-0.26
4,NLP,2011Q1,10.666,-3.392,10.666,12.967,177414.0,12.967,177414.0,48.098,...,177414.0,48.098,0.0,48.098,0.0,325.605,0.476,325.605,0.476,-0.30


# Rename columns for better readability

In [19]:
# Rename DataFrame columns to follow camelCase convention and match original definitions from the images

# Basic company identifiers
df = df.rename(columns={
    'tic': 'ticker',  # Stock ticker symbol
    'datafqtr': 'fiscalQuarter',  # Fiscal quarter date
    'conm': 'companyName'  # Company name
})

# Target variables (what we're trying to predict)
df = df.rename(columns={
    # Mean sale price per quarter (from additional variables)
    'SALE PRICE': 'salePrice'
})

# Basic financial variables (raw data from financial statements)
df = df.rename(columns={
    'cshoq': 'commonSharesOutstanding_',  # Number of common shares outstanding
    'cshtrq': 'commonSharesTraded_',  # Common shares traded
    'capxy': 'capitalExpenditure_',  # Funds used to acquire/upgrade physical assets

    'oiadpq': 'operatingIncome_',  # Operating income before depreciation
    'revtq': 'revenue_',  # Total revenue (sales) for the quarter

    'niq': 'netIncome_',  # Net income (profit after all expenses)
    'xoprq': 'operatingIncome_',  # Operating income after depreciation
    'teqq': 'shareholdersEquity_',  # Total shareholders' equity
    'atq': 'totalAssets_',  # Total assets on balance sheet
    'epsfiq': 'EPS_',  # Earnings per share (profit per outstanding share)
    'cogsq': 'costOfGoodsSold_'  # Cost of goods sold
})

# Engineered financial features (calculated metrics)
df = df.rename(columns={
    'Debt_to_TA': 'debtToTotalAssets',  # Total debt divided by total assets
    'NI_to_Asset': 'niToAsset',  # Net income divided by average assets
    'ROA': 'ROA',  # Return on assets (net income/total assets)
    'ROE': 'ROE',  # Return on equity (net income/shareholders' equity)
    'Revenue_growth': 'revenueGrowth',  # Quarter-over-quarter revenue growth
    'NI_growth': 'netIncomeGrowth',  # Quarter-over-quarter net income growth
    # Current ratio (current assets/current liabilities)
    'Current_ratio': 'currentRatio',
    # (Current assets - inventories)/current liabilities
    'Quick_ratio': 'quickRatio',
    'RSI': 'rsi',  # Relative Strength Index (momentum indicator)
    'MVA': 'mva',  # Market Value Added (not explicitly defined in images)
    'CFROI': 'cfroi',  # Cash flow return on investment (OCF/Capital Employed)

})

# df = df.rename(columns={
#     'gmrmexpq': 'roomsExpensesQuarterly',  # Rooms Expenses Quarterly
#     'gmrmrevq': 'roomRevenueQuarterly',  # Room Revenue Quarterly
#     'hbinvfhq': 'finishedHomesConstrInProgress',  # Finished Homes/Constr in Progress
#     'hbinvludq': 'landUnderDevelopment',  # Land Under Development
#     'hbinvtq': 'totalHomebuildingInventories',  # Total Homebuilding Inventories
#     'hbloq': 'homesitesLotsOwned'  # Homesites/Lots Owned
# })

# Macroeconomic variables (external factors)
df = df.rename(columns={
    'Real GDP SA(billion)': 'realGDPSA',  # Seasonally adjusted real GDP
    'M2 SA(billion)': 'm2SA',  # Seasonally adjusted M2 money supply
    # Difference between M2 and M1 money supply
    'M2-M1 SA(billion)': 'm2MinusM1SA',
    # Gap between M2 and M1 growth rates
    'M2-M1 Growth Gap(%)': 'm2m1GrowthGap',
    'M2V': 'm2Velocity',  # Velocity of M2 money supply
    'Prime Rate(%)': 'primeRate'  # Benchmark interest rate
})


# Drop unnecessary columns
df = df.drop(columns=['cshoq.1', 'cshtrq.1', 'capxy.1', 'oiadpq.1', 'niq.1', 'xoprq.1', 'teqq.1', 'atq.1', 'epsfiq.1',
                      'gmrmexpq',
                      'gmrmrevq',
                      'hbinvfhq',
                      'hbinvludq',
                      'hbinvtq',
                      'hbloq',
                      'sentiment_x',
                      'sentiment_y',
                      #   'm2m1GrowthGap',
                      #   'm2Velocity'
                      ])

print(df.columns.tolist())

['ticker', 'fiscalQuarter', 'commonSharesOutstanding_', 'commonSharesTraded_', 'capitalExpenditure_', 'operatingIncome_', 'netIncome_', 'operatingIncome_', 'shareholdersEquity_', 'totalAssets_', 'EPS_', 'companyName', 'revenue_', 'salePrice', 'realGDPSA', 'm2SA', 'm2m1GrowthGap', 'm2Velocity', 'primeRate']


# Company selection

In [20]:
nyc_real_estate_tickers = [
    'NEN',  # New York Mortgage Trust, Inc.
    'LEJUY',  # Leju Holdings Limited
    'MRCBF',   # Cushman & Wakefield plc
    'ACAN',  # Douglas Elliman Inc.
    'MDPCF',   # Jones Lang LaSalle Inc.
    'HMT.',  # Anywhere Real Estate Inc.
    'GYRO',  # RE/MAX Holdings, Inc.
    'PRLEQ',  # eXp World Holdings, Inc.
    'MYCB',  # Opendoor Technologies Inc.
    'TNL',  # Offerpad Solutions Inc.
    'MODVF',  # KE Holdings Inc. (Limited NYC presence)
    'ARL',  # Redfin Corporation
]
# Check if these tickers are in the DataFrame, which are not in the original list
for ticker in nyc_real_estate_tickers:
    if ticker not in df['ticker'].values:
        print(f"{ticker} is not in the DataFrame")
    else:
        print(f"{ticker} is in the DataFrame")

NEN is in the DataFrame
LEJUY is in the DataFrame
MRCBF is in the DataFrame
ACAN is in the DataFrame
MDPCF is in the DataFrame
HMT. is in the DataFrame
GYRO is in the DataFrame
PRLEQ is in the DataFrame
MYCB is in the DataFrame
TNL is in the DataFrame
MODVF is in the DataFrame
ARL is in the DataFrame


In [21]:
selected_companies = df[df['ticker'].isin(nyc_real_estate_tickers)]

# Process before the data cleaning phase
## Remove unnecessary columns

In [22]:
# Check for duplicate rows in the selected companies DataFrame
duplicate_rows = selected_companies[selected_companies.duplicated()]
if not duplicate_rows.empty:
    print("Duplicate rows in selected companies DataFrame:")
    print(duplicate_rows)
else:
    print("No duplicate rows in selected companies DataFrame")

No duplicate rows in selected companies DataFrame


## Check missing values

In [23]:
# Check numeric columns from the selected companies DataFrame
numeric_columns = selected_companies.select_dtypes(include=[np.number]).columns.tolist()
print("Numeric columns in selected companies DataFrame:")
print(numeric_columns)
# Check for missing values in the numeric columns
missing_numeric_values = selected_companies[numeric_columns].isnull().sum()
missing_numeric_values = missing_numeric_values[missing_numeric_values > 0]
if not missing_numeric_values.empty:
    print("Missing values in numeric columns of selected companies DataFrame:")
    print(missing_numeric_values)


Numeric columns in selected companies DataFrame:
['commonSharesOutstanding_', 'commonSharesTraded_', 'capitalExpenditure_', 'operatingIncome_', 'netIncome_', 'operatingIncome_', 'shareholdersEquity_', 'totalAssets_', 'EPS_', 'companyName', 'revenue_', 'salePrice', 'realGDPSA', 'm2SA', 'm2m1GrowthGap', 'm2Velocity', 'primeRate']
Missing values in numeric columns of selected companies DataFrame:
netIncome_             6
shareholdersEquity_    6
companyName            6
dtype: int64


# Preprocessing Steps

## Numerical features

### Process Outliers

In [24]:
from scipy.stats.mstats import winsorize

def handle_outliers(df, columns):
    """
    Handle outliers using the IQR method, clipping data to a reasonable range.
    """
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df[col] = df[col].clip(lower_bound, upper_bound)
    return df

    def handle_outliers_winsorize(df, columns, limits=(0.05, 0.05)):
        """
        Handle outliers using winsorization, which limits extreme values to a specified percentile.
        """
        for col in columns:
            df[col] = winsorize(df[col], limits=limits)
        return df

### Normalize numerical features

In [25]:
from sklearn.preprocessing import StandardScaler, RobustScaler

def scale_features(df, columns):
    """
    Standardize numerical features.
    """
    scaler = RobustScaler()
    df[columns] = scaler.fit_transform(df[columns])
    return df

### Utility function

In [26]:
def add_time_series_features(df):
    """
    Add time series features, such as lag values and time-related features.
    """
    df = df.sort_values(['ticker', 'fiscalQuarter'])
    df['EBITDA_lag1'] = df.groupby('ticker')['EBITDA'].shift(1)
    df['year'] = df['fiscalQuarter'].str[:4].astype(int)
    df['quarter'] = df['fiscalQuarter'].str[-1].astype(int)
    return df


def remove_multicollinearity(df, columns_to_drop):
    """
    Remove columns with high multicollinearity. Supports dropping multiple columns.
    """
    if isinstance(columns_to_drop, list):
        df = df.drop(columns=columns_to_drop)
    else:
        raise ValueError("columns_to_drop should be a list of column names.")
    return df

### Process missing (Null) values
<!--
EPS_                         9
commonSharesOutstanding_    24
commonSharesTraded_         54
quickRatio                  10
rsi                         64
mva                         70
cfroi                       10 -->

In [27]:
# Check for missing values in the DataFrame
# missing_values = selected_companies.isnull().sum()
# # Drop columns with missing values
# selected_companies = selected_companies.drop(columns=missing_values[missing_values > 0].index.tolist())
def process_missing_value(df, threshold=0.3):
    """
    Handle missing values: drop columns with missing ratio above the threshold
    and fill remaining missing values with the median.
    """
    missing_ratio = df.isnull().sum() / len(df)
    df = df.drop(columns=missing_ratio[missing_ratio > threshold].index)
    for col in df.select_dtypes(include=['float64', 'int64']):
        df[col].fillna(df[col].median(), inplace=True)
    return df

In [28]:
import numpy as np
import pandas as pd

def apply_log_transform(df, columns_to_transform, add_small_constant=True):
    """
    Apply logarithmic transformation to specified columns of a DataFrame (log(1 + x))
    to avoid issues with zero or negative values.

    Parameters:
    - df: Input DataFrame
    - columns_to_transform: List of column names to transform
    - add_small_constant: Whether to add 1 to the data (to avoid log(0))

    Returns:
    - Transformed DataFrame
    """
    df_transformed = df.copy()

    for col in columns_to_transform:
        if col not in df.columns:
            print(f"Warning: Column '{col}' does not exist, skipping")
            continue

        if add_small_constant:
            # Use log(1 + x) to avoid issues with zero or negative values
            df_transformed[col] = np.log1p(df_transformed[col])
        else:
            # Apply log only to positive values
            if (df_transformed[col] <= 0).any():
                print(f"Warning: Column '{col}' contains zero or negative values, consider using log(1 + x) or checking the data")
            df_transformed[col] = np.log(df_transformed[col])

    return df_transformed

In [29]:
from sklearn.preprocessing import LabelEncoder

def encode_categorical_features(df, columns):
    """
    Encode categorical features using LabelEncoder.

    Parameters:
    - df: Input DataFrame
    - columns: List of column names to encode

    Returns:
    - DataFrame with encoded columns
    """
    df_encoded = df.copy()
    for col in columns:
        encoder = LabelEncoder()
        df_encoded[col] = encoder.fit_transform(df_encoded[col])
    return df_encoded



In [30]:
selected_companies.head(3).to_csv("selected_companies.csv", index=False)

In [31]:
def preprocess_numeric_data(df):
    """
    Main function: call sub-functions to preprocess numerical data.
    """
    # 1. Handle missing values
    df = process_missing_value(df)

    # 2. Handle outliers
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    ratio_cols = ['debtToTotalAssets', 'niToAsset', 'ROA', 'EPS_', 'revenueGrowth', 'netIncomeGrowth', 'currentRatio', 
                  'm2m1GrowthGap', 'm2Velocity', 'primeRate'
                  ]  # Exclude ratio columns
    cols_to_process = list(set(numeric_cols) - set(ratio_cols))
    df = handle_outliers(df, cols_to_process)

    df = encode_categorical_features(df, ['ticker', 'companyName'])
# capitalExpenditure_ totalAssets_
# commonSharesOutstanding_
    log_transform_cols = [
    "commonSharesTraded_",  # Max 1.13B, mean 21.8M (highly skewed)
    "salePrice",            # Large monetary values
    # "totalAssets_",         # Max 122,520, mean 2,366 (right-skewed)
    "capitalExpenditure_",  # Max visible in stats, likely right-skewed
    "m2SA",                # Large monetary aggregate values
    "shareholdersEquity_"  # Max 46,740, mean 782 (highly skewed)

    ]
    df = apply_log_transform(df, log_transform_cols)

    #  Add time series features
    df = add_time_series_features(df)

    # 5. Scale features
    scale_columns = [
        "commonSharesOutstanding_",
        "netIncome_",
        "operatingIncome_",
        "costOfGoodsSold_",
        "realGDPSA"
    ]

    # ['debtToTotalAssets', 'niToAsset', 'ROA', 'currentRatio',
    #  'revenueGrowth', 'netIncomeGrowth', 'ebitdaGrowth',
    #  'ebitdaMargin', 'realGDPSA', 'm2SA']

    df = scale_features(df, scale_columns)

    return df


# Execute preprocessing
processed_df = preprocess_numeric_data(selected_companies.copy())

/var/folders/rm/zkx7tx614cq0pcwp0cpgbhy80000gn/T/ipykernel_36636/657722357.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col].fillna(df[col].median(), inplace=True)
/var/folders/rm/zkx7tx614cq0pcwp0cpgbhy80000gn/T/ipykernel_36636/657722357.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col].fillna(df[col].median(), inplace=True)


ValueError: Must specify axis=0 or 1

# Preprocessing Performance Comparison

## Verify outliers detection

In [ ]:
import matplotlib.pyplot as plt

def compare_outliers(df_before, df_after, column):
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.boxplot(df_before[column].dropna())
    plt.title(f"Before Outlier Handling: {column}")

    plt.subplot(1, 2, 2)
    plt.boxplot(df_after[column].dropna())
    plt.title(f"After Outlier Handling: {column}")

    plt.show()

compare_outliers(selected_companies, processed_df, 'capitalExpenditure_')


## Verify scaling performance

In [ ]:
import seaborn as sns
def check_scaling(df_before, df_after, column):
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    sns.histplot(df_before[column], kde=True)
    plt.title(f"Before Scaling: {column}")

    plt.subplot(1, 2, 2)
    sns.histplot(df_after[column], kde=True)
    plt.title(f"After Scaling: {column}")

    plt.show()

# 示例：检查totalAssets_的缩放效果
check_scaling(selected_companies, processed_df, 'totalAssets_')

## Check numerical features distribution after preprocessing

In [ ]:
def plot_distribution_comparison(df_before, df_after, columns, log_scale=False):
    for col in columns:
        plt.figure(figsize=(10, 4))

        # Distribution before processing
        plt.subplot(1, 2, 1)
        sns.histplot(df_before[col], kde=True, color='blue')
        if log_scale:
            plt.yscale('log')
        plt.title(f"Before: {col}")

        # Distribution after processing
        plt.subplot(1, 2, 2)
        sns.histplot(df_after[col], kde=True, color='green')
        if log_scale:
            plt.yscale('log')
        plt.title(f"After: {col}")

        plt.tight_layout()
        plt.show()

# Example: Check key numerical columns
numeric_cols = ['operatingIncome_', 'totalAssets_']
plot_distribution_comparison(selected_companies, processed_df, numeric_cols)

## Check correlation matrix

In [ ]:
def compare_correlation(df_before, df_after):
    plt.figure(figsize=(15, 6))

    # Drop the 'EBITDA' column before calculating correlations
    df_before_corr = df_before.drop(columns=['EBITDA']).select_dtypes(include=['float64']).corr()
    df_after_corr = df_after.drop(columns=['EBITDA']).select_dtypes(include=['float64']).corr()

    plt.subplot(1, 2, 1)
    sns.heatmap(df_before_corr, annot=False, cmap='coolwarm', vmin=-1, vmax=1)
    plt.title("Before Preprocessing")

    plt.subplot(1, 2, 2)
    sns.heatmap(df_after_corr, annot=False, cmap='coolwarm', vmin=-1, vmax=1)
    plt.title("After Preprocessing")

    plt.tight_layout()
    plt.show()

compare_correlation(selected_companies, processed_df)

# Save files

In [ ]:
processed_df.to_csv("revenue_new.csv", index=False)
print("New CSV file created: revenue_new.csv")

In [ ]:
# 打印每一列的数据统计
print(df.describe(include='all'))

In [ ]:
# 打印 EBITDA 列的统计信息
print(df['EBITDA'].describe())

In [ ]:
# 打印EBITDA的数值分布
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10, 6))
sns.histplot(df['EBITDA'], bins=30, kde=True)
plt.title('EBITDA Distribution')
plt.xlabel('EBITDA')
plt.ylabel('Frequency')
plt.show()

done
<!--
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# 选择特征和目标变量
features = ['capitalExpenditure_', 'netIncome_', 'operatingIncome_', 'shareholdersEquity_',
            'totalAssets_', 'EPS_', 'debtToTotalAssets', 'niToAsset', 'ROA', 'revenueGrowth',
            'quickRatio', 'rsi', 'mva', 'cfroi', 'salePrice', 'realGDPSA', 'm2SA',
            'm2MinusM1SA', 'm2Velocity', 'primeRate', 'EBITDA_lag1']
target = 'EBITDA'

# 去除缺失值
processed_df = processed_df.dropna(subset=features + [target])

# 数据归一化
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(processed_df[features + [target]])

# 构造时间序列数据
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, :-1])
        y.append(data[i+seq_length, -1])
    return np.array(X), np.array(y)

seq_length = 10  # 时间步长
X, y = create_sequences(scaled_data, seq_length)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 构建LSTM模型
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(1)
])

# 编译模型
model.compile(optimizer='adam', loss='mean_squared_error')

# 训练模型
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# 预测
y_pred = model.predict(X_test)

# 反归一化预测值和真实值
y_test_rescaled = scaler.inverse_transform(np.hstack((np.zeros((y_test.shape[0], len(features))), y_test.reshape(-1, 1))))[:, -1]
y_pred_rescaled = scaler.inverse_transform(np.hstack((np.zeros((y_pred.shape[0], len(features))), y_pred)))[:, -1]

# 评估模型性能
mse = mean_squared_error(y_test_rescaled, y_pred_rescaled)
r2 = r2_score(y_test_rescaled, y_pred_rescaled)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}") -->